In [ ]:
from geomdl import BSpline
from geomdl import utilities

import numpy as np
import math

from scipy import optimize
from scipy.optimize import minimize, NonlinearConstraint
from scipy.stats import multivariate_normal

In [ ]:
# FOV_range = 10
# scale_resolution = .1
# x_ = np.arange(0, FOV_range*2, scale_resolution)
# y_ = np.arange(0, FOV_range*2, scale_resolution)
# z_ = np.arange(0, FOV_range*2, scale_resolution)
# x0 = 0
# y0 = 0
# sigx = 1
# sigy = 1
# Xg, Yg = np.meshgrid(x_, y_)
# SiP = [0, 4]
obj = 4
probability_threshold = 0.005
# numbers_of_ctrl_points = math.ceil(20*1.7)
numbers_of_ctrl_points = 20

In [ ]:
int_mean = np.array([0.0, 0.0, 0.0])
# print(mean)
# mean += np.random.normal(0, 1e-3, mean.shape) # add small noise to avoid singular matrix
# mean[2] += np.random.normal(0, 1e-1, mean[2].shape) # add small noise to avoid singular
# print(mean)
cov = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
x = np.linspace(-0.0, 20.0, 40)
y = np.linspace(-0.0, 20.0, 40)
z = np.linspace(-0.0, 20.0, 40)
X, Y, Z = np.meshgrid(x, y, z)
pos = np.empty(X.shape + (3,))
pos[:, :, :, 0] = X
pos[:, :, :, 1] = Y
pos[:, :, :, 2] = Z

In [ ]:
Start_point = (0.0, 0.0, 0.0) #startpoint
Goal_point = (20.0, 20.0, 0.0) #goal position

In [ ]:
def inturder_linear_path(start, stop, num):
    '''Calculate the linear path between two points'''
    x = np.linspace(start[0], stop[0], num)
    y = np.linspace(start[1], stop[1], num)
    z = np.linspace(start[2], stop[2], num)
    return x, y, z

In [ ]:
inturder_list = []

local = inturder_linear_path([0.0, 15.0, 0], [16, 10, 0.0], numbers_of_ctrl_points)
inturder_list.append(local)
local = inturder_linear_path([20, 5.0, 0], [0, 10, 0], numbers_of_ctrl_points)
inturder_list.append(local)
local = inturder_linear_path([2.5, 0, 0], [20, 17.5, 0], numbers_of_ctrl_points)
inturder_list.append(local)
# print(inturder_list)
print(len(inturder_list))

In [ ]:
def object_function(x):
    '''Calculate the difference between the goal position and the current
    position of the distance'''
    res = math.dist((x[-3], x[-2], x[-1]), Goal_point)
    # print(res)
    return res

In [ ]:
def con_cltr_pnt(x):
    con = []
    con.append(math.dist((Start_point[0], Start_point[1], Start_point[2]), (x[0], x[1], x[2])))
    for i in range(0, len(x)-3, 3):
        con.append(math.dist([x[i], x[i+1], x[i+2]], [x[i+3], x[i+4], x[i+5]]))
    return con

C_nlc = NonlinearConstraint(con_cltr_pnt, 0.0, 1.0)

In [ ]:
def con_pdf_muti(x):
    tmp = []

    for i in range(0, len(x), 3):
        out = 0
        for j in range(len(inturder_list)):
            mean = np.array([inturder_list[j][0][int(i/3)], inturder_list[j][1][int(i/3)], inturder_list[j][2][int(i/3)]])
            # print(mean)
            rv = multivariate_normal(mean, cov)
            W = rv.pdf([x[i], x[i+1], x[i+2]])
            out = out + W
            
        tmp.append(out)
        # print(out)
    return tmp

P_muti_nlc = NonlinearConstraint(con_pdf_muti, 0.0, probability_threshold)

In [ ]:
int_X0 = np.zeros(numbers_of_ctrl_points*3)
# res = minimize(object_function, int_X0, constraints=[C_nlc, P_muti_nlc])
res = minimize(object_function, int_X0, method='SLSQP', bounds=[(-0, 20) for i in range(len(int_X0))], constraints=[C_nlc, P_muti_nlc])

print(res.success)
print(res.message)
print(len(res.x))

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly
plotly.offline.init_notebook_mode()

In [ ]:
# Create a B-Spline curve
curve = BSpline.Curve()
# Set up the curve
curve.degree = 4

x=[]
y=[]
z=[]
x.append(Start_point[0])
y.append(Start_point[1])
z.append(Start_point[2])
for i in range(0, len(res.x), 3):
    # print(res.x[i], res.x[i+1], res.x[i+2])
    x.append(res.x[i])
    y.append(res.x[i+1])
    z.append(res.x[i+2])

cp = np.array([x, y, z]).T.tolist()
# print(cp)
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    name='Control Points',
    marker = dict(
        size = 5
    ),
    opacity=0.5
)
# plt.show()
curve.ctrlpts = cp
# Auto-generate knot vector
curve.knotvector = utilities.generate_knot_vector(curve.degree, len(curve.ctrlpts))
# Set evaluation delta
curve.delta = 0.01
curvepts = np.array(curve.evalpts)
# curveplt, = plt.plot(curvepts[:, 0], curvepts[:, 1], color='yellowgreen', linestyle='-')

trace2 = go.Scatter3d(
    x=curvepts[:, 0], 
    y=curvepts[:, 1], 
    z=curvepts[:, 2], 
    mode='lines',
    name='BSpline'
)

SToE = go.Scatter3d(
    x=[Start_point[0], Goal_point[0]], 
    y=[Start_point[1], Goal_point[1]],
    z=[Start_point[2], Goal_point[2]],
    mode='lines',
    line=dict(dash='dash', color='black'),
    name='Start To End',
)

rv = multivariate_normal(int_mean, cov)
out_pdf = rv.pdf(pos)

gau = go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=out_pdf.flatten(),
    isomin=probability_threshold,
    # isomax=0.7,
    opacity=0.3,
    surface_count=32,
    slices_z=dict(show=True, locations=[10.0]),
    surface=dict(fill=0.2, pattern='odd'), 
    caps= dict(x_show=False, y_show=False, z_show=False), # no caps
    colorscale='turbo') # color scale

layout = go.Layout(
    title='3D',
    # scene=dict(
    #         camera= dict(
    #             up=dict(x=0, y=0, z=1),
    #             center=dict(x=0, y=0, z=0),
    #             eye=dict(x=1, y=-1, z=0)
    #         )
    # )
)

fig = go.Figure(data=gau, layout=layout)
# fig.add_volume(gau)
# fig = go.Figure(data=[go.Scatter3d(x=curvepts[:, 0], y=curvepts[:, 1], z=curvepts[:, 2], mode='lines')], layout=layout)
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(SToE)
fig.update_layout(height=800, width=960,
                  legend=dict(
                            orientation="h",
                            # entrywidth=500,
                            # yanchor="bottom",
                            # y=1.02,
                            # xanchor="right",
                            # x=1
                            )
                  )


plotly.offline.iplot(fig, filename='simple-3d-scatter')

# fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly
plotly.offline.init_notebook_mode()

data = []
frames = []
# make frames

for i in range(0, len(res.x), 3):
    out = 0

    for j in range(len(inturder_list)):
        # print(i, j)
        # gau = gauss_function_2d([x[i], x[i+1]], inturder_list[j][0][int(i/2)], inturder_list[j][1][int(i/2)], 1.05**(i/2), 1.05**(i/2))
        intruder_mean = np.array([inturder_list[j][0][int(i/3)], inturder_list[j][1][int(i/3)], inturder_list[j][2][int(i/3)]])
        # print(intruder_mean)
        rv = multivariate_normal(intruder_mean, cov)
        W = rv.pdf(pos)
        out = out + W
    
    data_dict = go.Volume(  x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
                            value=out.flatten(),
                            # isomin=probability_threshold,
                            # isomax=0.7,
                            # opacityscale="max",
                            # opacityscale="uniform",
                            opacity=0.3,
                            # opacityscale=[[0, 0], [0.06, 1]],
                            surface_count=32,
                            # slices_z=dict(show=True, locations=[10.0]),
                            surface=dict(fill=0.2, pattern='odd'), 
                            caps= dict(x_show=False, y_show=False, z_show=False), # no caps
                            colorscale='turbo') # color scale

    frames.append(go.Frame(data=[data_dict])) 
print(len(frames))
layout=go.Layout(
                height=800, width=960,
                legend=dict(
                            orientation="h",
                            ),
                updatemenus=[dict(type="buttons",
                             buttons=[dict(label="Play",
                                    method="animate",
                                    args=[None])])]
                )

fig = go.Figure(data=data_dict,
                layout=layout,
                frames=frames,
                )

# plotly.offline.iplot(fig, filename='Volume3d')
# fig.show()
fig.write_html('Volume3d32.html')

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot  # for IDE use

vol4 = np.random.randint(0, 27, size=(6, 5, 7, 4))

a, b, c = vol4.shape[1:]
X, Y, Z = np.mgrid[-1:1:a*1j, -1:1:b*1j, -1:1:c*1j]

fig = go.Figure(go.Volume(x=X.flatten(),
                          y=Y.flatten(),
                          z=Z.flatten(),
                          value=vol4[0].flatten(),
                          opacity=.25,
                          surface_count=12,
                          colorscale='turbo',
                          colorbar_len=0.8
    ))

frames=[go.Frame(data=go.Volume(
                               value=vol4[k].flatten()), 
                 name=str(k)) for k in range(len(vol4))]

updatemenus = [dict(
        buttons = [
            dict(
                args = [None, {"frame": {"duration": 100, "redraw": True},
                                "fromcurrent": True, "transition": {"duration": 0}}],
                label = "Play",
                method = "animate"
                ),
            dict(
                 args = [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                label = "Pause",
                method = "animate"
                )
        ],
        direction = "left",
        pad = {"r": 10, "t": 87},
        showactive = False,
        type = "buttons",
        x = 0.21,
        xanchor = "right",
        y = -0.075,
        yanchor = "top"
    )] 

sliders = [dict(steps = [dict(method= 'animate',
                              args= [[f'{k}'],                           
                              dict(mode= 'immediate',
                                   frame= dict(duration=100, redraw=True),
                                   transition=dict(duration= 0))
                                 ],
                              label=f'{k+1}'
                             ) for k in range(len(vol4))], 
                active=0,
                transition= dict(duration=0),
                x=0, # slider starting position  
                y=0, 
                currentvalue=dict(font=dict(size=12), 
                                  prefix='frame: ', 
                                  visible=True, 
                                  xanchor= 'center'
                                 ),  
                len=1.0) #slider length
           ]

fig.update_layout(width=700, height=700, updatemenus=updatemenus, sliders=sliders)
fig.update(frames=frames)
# plot(fig, auto_open=True)